In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
import const

# on importe le dossier contenant les classes à tester
path_test = os.path.join(const.DATASET_CLEAN_WO_BACKGROUND, 'test')


def modelArchitecture(n_class):

    # on importe le modèle de base
    base_model = EfficientNetB7(weights='imagenet', include_top=False) 

    # on freeze le modèle de base
    for layer in base_model.layers: 
        layer.trainable = False

    # on créer notre modèle
    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D()) 
    model.add(Dense(1024,activation='relu'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(n_class, activation='softmax'))

    # on défreeze les couches
    for layer in base_model.layers[-20:]:
        layer.trainable = True

    return model

def modelArchitectureYoni(n_class):

    # même process qu'avant mais avec l'architecture de Yoni

    base_model = EfficientNetB0(weights='imagenet', include_top=False)
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(rate=0.2)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(rate=0.2)(x)
    predictions = Dense(n_class, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers[-10:]:
        layer.trainable = True

    return model

# il faut appeler la fonction correspondante et indiquer le nombre de classes de la première moitié
model = modelArchitectureYoni(262)

# on importe les poids du premier modèle
model.load_weights('../prod_script/models/model_yoni_B0_half_1.h5')

# on créer le générateur pour test
test_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)

# on charge les images et on les redimensionne
test_generator = test_data_generator.flow_from_directory(
    directory=path_test,
    target_size=(224, 224),
    batch_size=92,
    class_mode='sparse',
    shuffle=False
)

# on effectue les prédictions
predictions = model.predict(test_generator)

# on stocke dans une liste les classes prédites au lieu des probabilités
pred_classes_1 = np.argmax(predictions, axis=1)

# on stocke les probabilités qui vont avec les classes retenues
pred_probas_1 = np.max(predictions, axis=1)

# On répète le processus
model = modelArchitectureYoni(263)
model.load_weights('../prod_script/models/model_yoni_B0_half_2.h5')

predictions = model.predict(test_generator)
pred_classes_2 = np.argmax(predictions, axis=1)
pred_probas_2 = np.max(predictions, axis=1)



Found 11025 images belonging to 525 classes.
120/120 [==============================] - 16s 128ms/step


In [2]:

# on fais une liste complète de toutes classes prédites
classes_list = []
for proba_1, proba_2, classe_1, classe_2 in zip(pred_probas_1, pred_probas_2, pred_classes_1, pred_classes_2):

    # on choisit la classe qui a la meilleur probabilité entre les deux modèles
    if proba_1 > proba_2:
        classes_list.append(classe_1)
    elif proba_2 >= proba_1:
        classes_list.append(classe_2 + 262) # il faut bien ajouter le nombre de classes de la première partie ici

print(classes_list)

[0, 0, 0, 0, 0, 0, 520, 164, 0, 0, 0, 0, 32, 519, 283, 86, 0, 283, 0, 10, 0, 1, 1, 344, 1, 1, 1, 1, 43, 1, 1, 301, 378, 1, 1, 110, 37, 403, 135, 1, 9, 53, 2, 2, 2, 2, 2, 2, 2, 2, 2, 58, 2, 2, 2, 2, 335, 7, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 509, 4, 509, 517, 4, 368, 4, 4, 284, 4, 459, 4, 4, 4, 4, 4, 4, 4, 4, 4, 434, 5, 5, 5, 5, 5, 5, 5, 5, 5, 429, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 297, 6, 6, 6, 375, 375, 6, 6, 6, 375, 375, 6, 375, 375, 6, 6, 375, 6, 6, 7, 7, 213, 7, 7, 7, 499, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 296, 7, 7, 8, 8, 8, 8, 85, 8, 411, 434, 8, 8, 8, 8, 8, 8, 207, 8, 8, 8, 8, 8, 8, 9, 9, 344, 511, 9, 9, 9, 9, 9, 357, 344, 9, 9, 9, 9, 9, 9, 508, 9, 358, 9, 382, 10, 10, 10, 10, 440, 10, 10, 10, 77, 10, 10, 519, 196, 10, 10, 10, 10, 10, 259, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 180, 12, 12, 351, 12, 12, 12, 12, 12, 12, 351, 297, 12, 12, 12, 486, 12, 12, 465, 12, 13, 205, 13, 13, 1

In [3]:
from sklearn.metrics import classification_report

# on récupère les vrais labels
true_labels = test_generator.classes

print(classification_report(true_labels, classes_list))

              precision    recall  f1-score   support

           0       0.81      0.62      0.70        21
           1       0.85      0.52      0.65        21
           2       1.00      0.86      0.92        21
           3       0.91      1.00      0.95        21
           4       0.82      0.67      0.74        21
           5       1.00      0.95      0.98        21
           6       1.00      0.62      0.76        21
           7       0.82      0.86      0.84        21
           8       1.00      0.81      0.89        21
           9       0.79      0.71      0.75        21
          10       0.88      0.71      0.79        21
          11       1.00      1.00      1.00        21
          12       0.94      0.71      0.81        21
          13       1.00      0.57      0.73        21
          14       1.00      0.95      0.98        21
          15       1.00      0.81      0.89        21
          16       0.95      0.86      0.90        21
          17       0.95    

In [4]:
# on en génère un deuxième avec en sortie un dictionnaire
report_dict = classification_report(true_labels, classes_list, output_dict = True)

# on récupère les 20 classes qui ont le pire score F1

# création d'un dictionnaire vide pour contenir les scores des classes
class_scores = {}

# on supprime les clés non utiles
for cle, valeur in report_dict.items():
    if cle not in ['accuracy', 'macro avg', 'weighted avg']:
        class_scores[cle] = valeur

# liste pour stocker les classes et leurs scores F1
tri_classes = []

# on ajoute un tuple avec à chaque fois la classe et le score F1
for label_classe, metrics in class_scores.items():
    tri_classes.append((label_classe, round(metrics['f1-score'], 2)))

# on trie la liste dans l'ordre des scores F1
tri_classes.sort(key = lambda x: x[1])

# on affiche les 20 premières entrées de la liste
pires_classes = tri_classes[:20]

# on parcours la liste pour afficher les informations
for label_classe, score_f1 in pires_classes:
    print(f"Classe : {label_classe}, score F1: {score_f1}")

Classe : 129, score F1: 0.3
Classe : 43, score F1: 0.33
Classe : 92, score F1: 0.38
Classe : 241, score F1: 0.44
Classe : 481, score F1: 0.47
Classe : 56, score F1: 0.5
Classe : 147, score F1: 0.56
Classe : 25, score F1: 0.61
Classe : 236, score F1: 0.61
Classe : 251, score F1: 0.61
Classe : 1, score F1: 0.65
Classe : 18, score F1: 0.65
Classe : 86, score F1: 0.67
Classe : 108, score F1: 0.67
Classe : 109, score F1: 0.67
Classe : 224, score F1: 0.67
Classe : 430, score F1: 0.67
Classe : 450, score F1: 0.68
Classe : 44, score F1: 0.69
Classe : 143, score F1: 0.69
